In [1]:
import sys
print(sys.executable)

E:\Program Files (x86)\Anaconda3\envs\dissertation\python.exe


In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np


sys.path.append('E:\GitHubProjects\dissertation\Scripts')
import helperfn as hf

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
from gensim.models import KeyedVectors

In [4]:
embedding_model = KeyedVectors.load_word2vec_format(r'E:\GitHubProjects\dissertation\word2vec\GoogleNews-vectors-negative300.bin', binary=True)
# If we don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
embedding_model.init_sims(replace=True)

In [7]:
#Get Data
data = hf.merge_datasets(r'E:\GitHubProjects\dissertation\scraper\approved_datasets')

#Clean reivews (X)

# #Remove stop words
# X = X.apply(lambda x: hf.remove_punctuations(x))

#Tokenizing the text using NLTK
#Tokenize [TODO dont split apostrophe e.g. don't but students' ] 
# TODO ******. doesnt segment full stop 
# tokenizer = RegexpTokenizer(pattern='\w+|\$[\d\.]+|\S+')
# tokenizer = RegexpTokenizer(r'\w+')
# https://stackoverflow.com/questions/42056872/how-to-remove-in-strings-with-regexptokenizer
tokenizer = RegexpTokenizer(r'[\w\']+') #not seperating apostrophe


data['tokenized_text'] = data['review'].apply(lambda x: tokenizer.tokenize(x)) 

#lower case
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: hf.lower_token(x))


In [ ]:
model = Sequential()

# Embedding Layer, output dimension is 300 because word2vec 300d
embedding_layer = Embedding(vocab_size, embedding_vecor_length, weights=[embedding_matrix], input_length=max_review_length), trainable=True)

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)


model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Display summary of the model
print(model.summary())

# Fit the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          shuffle=True,
          epochs=nb_epochs,
          validation_data=(X_test, Y_test),
          callbacks=[EarlyStopping(min_delta=0.00025, patience=2)])




# model_cnn = Sequential()
# e = Embedding(100000, 100, input_length=1000)
# model_cnn.add(e)
# model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
# model_cnn.add(GlobalMaxPooling1D())
# model_cnn.add(Dense(256, activation='relu'))
# model_cnn.add(Dense(1, activation='sigmoid'))
# model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_cnn.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
# score,acc = model_cnn.evaluate(x_val_seq, y_validation, verbose = 2, batch_size = 32)
# print("score: %.2f" % (score))
# print("acc: %.2f" % (acc))


In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

#Get Data
data = hf.merge_datasets(r'E:\GitHubProjects\dissertation\scraper\approved_datasets')
# data = hf.balance_dataset(data)

#Clean reivews (X)

# #Remove stop words
# X = X.apply(lambda x: hf.remove_punctuations(x))

#Tokenizing the text using NLTK
#Tokenize [TODO dont split apostrophe e.g. don't but students' ] 
# TODO ******. doesnt segment full stop 
# https://stackoverflow.com/questions/42056872/how-to-remove-in-strings-with-regexptokenizer
tokenizer = RegexpTokenizer(r'[\w\']+') #not seperating apostrophe


data['tokenized_text'] = data['review'].apply(lambda x: tokenizer.tokenize(x)) 

#remove stop words
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: [item for item in x if item not in stop])

#lower case
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: hf.lower_token(x))

In [6]:
# X, y = list(data['review']), list(data['score'])
X, y = data['tokenized_text'], list(data['score'])

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#vocab size?
NUM_WORDS = 10000
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [8]:
#finding the maximum number of words the longests sentence contains
# length = []
# # TODO X_train might need to be a list not dataframe ["aa", "bb"]
# for x in X_train:
#     length.append(len(x.split()))
# max(length)

#my more memory efficient version
# longest = 0
# for x in X_train:
#     if len(x.split()) > longest:
#         longest = len(x.split())

#VERSION FOR TOKENIZED ROWS
longest = 0
for x in X:
    if len(x) > longest:
        longest = len(x)


#padding sequences to have the same length
x_train_seq = pad_sequences(sequences, maxlen=longest+5)

In [36]:
sequences_val = tokenizer.texts_to_sequences(X_test)
x_val_seq = pad_sequences(sequences_val, maxlen=longest+5)

In [50]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(x_train_seq, y, test_size=0.33, shuffle=True)

In [51]:
#Word Embeddings
#length of vectors in embedding word2vec
embedding_vecor_length = 300
embedding_matrix = np.zeros((NUM_WORDS, embedding_vecor_length))

for word, i in tokenizer.word_index.items():
    if i >= NUM_WORDS:
        continue
    try:
        embedding_vector = embedding_model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        pass
        # embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25), embedding_vecor_length)

In [52]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import MaxPooling1D


In [56]:
model_cnn = Sequential()
e = Embedding(NUM_WORDS, 300, weights=[embedding_matrix], input_length=longest+5, trainable=False)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=9, batch_size=32, verbose=2)

Train on 1095 samples, validate on 540 samples
Epoch 1/9
 - 0s - loss: 0.5941 - accuracy: 0.6584 - val_loss: 0.4361 - val_accuracy: 0.8537
Epoch 2/9
 - 0s - loss: 0.3293 - accuracy: 0.8913 - val_loss: 0.2789 - val_accuracy: 0.8870
Epoch 3/9
 - 0s - loss: 0.1923 - accuracy: 0.9315 - val_loss: 0.2475 - val_accuracy: 0.8944
Epoch 4/9
 - 0s - loss: 0.1116 - accuracy: 0.9680 - val_loss: 0.2503 - val_accuracy: 0.9093
Epoch 5/9
 - 0s - loss: 0.0610 - accuracy: 0.9945 - val_loss: 0.2457 - val_accuracy: 0.8981
Epoch 6/9
 - 0s - loss: 0.0324 - accuracy: 0.9963 - val_loss: 0.2580 - val_accuracy: 0.8963
Epoch 7/9
 - 0s - loss: 0.0174 - accuracy: 0.9973 - val_loss: 0.2698 - val_accuracy: 0.9037
Epoch 8/9
 - 0s - loss: 0.0113 - accuracy: 0.9991 - val_loss: 0.2792 - val_accuracy: 0.9019
Epoch 9/9
 - 0s - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.2900 - val_accuracy: 0.9056


In [57]:
# tk_test = Tokenizer()
# tk_test.fit_on_texts(X_test)
# index_list = tk_test.texts_to_sequences(X_test)
# x_train_padded = pad_sequences(index_list, maxlen=longest+5)

score = model_cnn.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.29001467161708405 / Test accuracy: 0.9055555462837219


In [62]:
from sklearn.metrics import confusion_matrix

y_pred = (model_cnn.predict(X_test).ravel()>0.5)+0 # predict and get class (0 if pred < 0.5 else 1)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
confusion_matrix(y_test, y_pred)

array([[149,  26],
       [ 25, 340]], dtype=int64)

In [61]:
print("Precision: ", tp / (tp+fp))
print("Recall: ", tp / (tp+fn))

Precision:  0.9289617486338798
Recall:  0.9315068493150684


In [63]:
tn

149